## BDA Final Project
I cook the JSON.

In [77]:
# ⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⠿⠿⠿⠿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⣿⠟⠋⠁⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢺⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠜⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⠿⠿⠛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠻⣿⣿⣿⣿⣿
# ⣿⣿⡏⠁⠀⠀⠀⠀⠀⣀⣠⣤⣤⣶⣶⣶⣶⣶⣦⣤⡄⠀⠀⠀⠀⢀⣴⣿⣿⣿⣿⣿
# ⣿⣿⣷⣄⠀⠀⠀⢠⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢿⡧⠇⢀⣤⣶⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣾⣮⣭⣿⡻⣽⣒⠀⣤⣜⣭⠐⢐⣒⠢⢰⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⣏⣿⣿⣿⣿⣿⣿⡟⣾⣿⠂⢈⢿⣷⣞⣸⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⣿⣽⣿⣿⣷⣶⣾⡿⠿⣿⠗⠈⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿⠻⠋⠉⠑⠀⠀⢘⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⣿⡿⠟⢹⣿⣿⡇⢀⣶⣶⠴⠶⠀⠀⢽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⣿⣿⣿⡿⠀⠀⢸⣿⣿⠀⠀⠣⠀⠀⠀⠀⠀⡟⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
# ⣿⣿⣿⡿⠟⠋⠀⠀⠀⠀⠹⣿⣧⣀⠀⠀⠀⠀⡀⣴⠁⢘⡙⢿⣿⣿⣿⣿⣿⣿⣿⣿
# ⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⢿⠗⠂⠄⠀⣴⡟⠀⠀⡃⠀⠉⠉⠟⡿⣿⣿⣿⣿
# ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢷⠾⠛⠂⢹⠀⠀⠀⢡⠀⠀⠀⠀⠀⠙⠛⠿⢿

In [78]:
import pandas as pd

# Load the JSON file into a Pandas DataFrame
salesdf = pd.read_json('mongodb_sample_supplies.json', lines=True)

In [79]:
salesdf.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"[{'name': 'laptop', 'tags': ['electronics', 's...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"[{'name': 'printer paper', 'tags': ['office', ...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online
2,{'$oid': '5bd761dcae323e45a93ccfeb'},{'$date': '2015-02-23T09:53:59.343Z'},"[{'name': 'binder', 'tags': ['school', 'genera...",Seattle,"{'gender': 'F', 'age': 45, 'email': 'vatires@t...",False,In store
3,{'$oid': '5bd761dcae323e45a93ccfe8'},{'$date': '2015-03-23T21:06:49.506Z'},"[{'name': 'printer paper', 'tags': ['office', ...",Denver,"{'gender': 'M', 'age': 42, 'email': 'cauho@wit...",True,Online
4,{'$oid': '5bd761dcae323e45a93ccfe9'},{'$date': '2015-08-25T10:01:02.918Z'},"[{'name': 'envelopes', 'tags': ['stationary', ...",Seattle,"{'gender': 'M', 'age': 50, 'email': 'keecade@h...",False,Phone


In [80]:
#Flatten the 'items' column into individual rows
salesdf = salesdf.explode('items')


In [81]:
salesdf.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'laptop', 'tags': ['electronics', 'sc...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'envelopes', 'tags': ['stationary', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'printer paper', 'tags': ['office', '...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'notepad', 'tags': ['office', 'writin...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online


### Query 1: Show top 10 products (name) sales (quantity x price).

In [82]:
# Get the product name from 'items' dictionary
# Since the product name is nested in a dictionary, we need to use a lambda function to extract it
# The 'name' is stored as a dictionary with a key 'items' within each row
# we need to access the # value of 'items'

# Extract the 'product_name' from the 'items' dictionaries, which are nested within each row. 
# Access the 'name' value within these dictionaries as follows:

salesdf['product_name'] = salesdf['items'].apply(lambda x: x['name'])

In [83]:
print(salesdf.columns)

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'product_name'],
      dtype='object')


In [84]:
salesdf.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,product_name
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'laptop', 'tags': ['electronics', 'sc...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,laptop
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'envelopes', 'tags': ['stationary', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,envelopes
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,printer paper
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'printer paper', 'tags': ['office', '...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online,printer paper
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'notepad', 'tags': ['office', 'writin...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online,notepad


In [85]:
# Group by product name and count the occurrences
top_products = salesdf['product_name'].value_counts().nlargest(10).reset_index()

# Rename the columns for clarity
top_products.columns = ['Product Name', 'Count']

print(top_products)

    Product Name  Count
0        notepad   6872
1         binder   4676
2           pens   4559
3      envelopes   4505
4       backpack   2309
5         laptop   2271
6  printer paper   2246


### Query 2: Show top 3 products (name) sales by store (location).

In [86]:
# Extract the 'quantity' and 'price' from the 'items' dictionaries, which are nested within each row. 
# Access the 'quantity' and 'price' value within these dictionaries as follows:
# Calculate the sales (quantity x price) for each product

salesdf['sales'] = salesdf['items'].apply(lambda x: x['quantity'] * float(x['price']['$numberDecimal']))


In [87]:
salesdf.columns

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'product_name', 'sales'],
      dtype='object')

In [88]:
salesdf.tail()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,product_name,sales
4998,{'$oid': '5bd761deae323e45a93ce35a'},{'$date': '2015-04-27T23:12:34.004Z'},"{'name': 'notepad', 'tags': ['office', 'writin...",Austin,"{'gender': 'M', 'age': 41, 'email': 'bi@kok.pw...",True,In store,notepad,70.88
4999,{'$oid': '5bd761deae323e45a93ce36a'},{'$date': '2013-10-06T22:39:37.868Z'},"{'name': 'envelopes', 'tags': ['stationary', '...",Denver,"{'gender': 'F', 'age': 62, 'email': 'opujuw@ab...",False,In store,envelopes,163.80
4999,{'$oid': '5bd761deae323e45a93ce36a'},{'$date': '2013-10-06T22:39:37.868Z'},"{'name': 'binder', 'tags': ['school', 'general...",Denver,"{'gender': 'F', 'age': 62, 'email': 'opujuw@ab...",False,In store,binder,79.74
4999,{'$oid': '5bd761deae323e45a93ce36a'},{'$date': '2013-10-06T22:39:37.868Z'},"{'name': 'envelopes', 'tags': ['stationary', '...",Denver,"{'gender': 'F', 'age': 62, 'email': 'opujuw@ab...",False,In store,envelopes,21.48
4999,{'$oid': '5bd761deae323e45a93ce36a'},{'$date': '2013-10-06T22:39:37.868Z'},"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'F', 'age': 62, 'email': 'opujuw@ab...",False,In store,printer paper,84.88


In [89]:
# Group by store location and product name, and sum the sales
top_3_products_by_store = salesdf.groupby(['storeLocation', salesdf['items'].apply(lambda x: x['name'])])['sales'].sum().reset_index()

# Rank the products by sales for each store location
top_3_products_by_store['rank'] = top_3_products_by_store.groupby('storeLocation')['sales'].rank(ascending=False, method='min')

# Filter the top 3 products for each store location
top_3_products_by_store = top_3_products_by_store[top_3_products_by_store['rank'] <= 3]

print(top_3_products_by_store)

   storeLocation     items       sales  rank
0         Austin  backpack   115217.59   2.0
3         Austin    laptop  1018494.05   1.0
5         Austin      pens    74622.52   3.0
7         Denver  backpack   245679.34   2.0
10        Denver    laptop  1961659.25   1.0
12        Denver      pens   185008.39   3.0
14        London  backpack   144529.14   2.0
17        London    laptop  1073098.72   1.0
19        London      pens    92602.30   3.0
21      New York  backpack    82563.14   2.0
24      New York    laptop   694613.32   1.0
26      New York      pens    64029.90   3.0
28     San Diego  backpack    50997.42   2.0
31     San Diego    laptop   462973.58   1.0
33     San Diego      pens    42134.55   3.0
35       Seattle  backpack   178387.47   2.0
38       Seattle    laptop  1565138.15   1.0
40       Seattle      pens   123445.61   3.0


### Query 3: Show rankings of each store (location).

In [90]:
# Group by store location and sum the sales for each location
store_rankings = salesdf.groupby('storeLocation')['sales'].sum().reset_index()

# Rank the store locations by sales
store_rankings['rank'] = store_rankings['sales'].rank(ascending=False, method='min')

# Sort the store rankings by rank
store_rankings = store_rankings.sort_values(by='rank')

print(store_rankings)

  storeLocation       sales  rank
1        Denver  2921009.92   1.0
5       Seattle  2255947.69   2.0
2        London  1583066.79   3.0
0        Austin  1445603.11   4.0
3      New York  1016059.59   5.0
4     San Diego   672885.17   6.0


In [91]:
salesdf.columns

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'product_name', 'sales'],
      dtype='object')

### Query 4: Show purchased method by gender in a table

In [92]:
# Extract the 'gender' from the 'cusomter' dictionaries, which are nested within each row. 
# Access the 'gender' value within these dictionaries as follows:

salesdf['gender'] = salesdf['customer'].apply(lambda x: x.get('gender'))


In [93]:
salesdf.columns

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'product_name', 'sales', 'gender'],
      dtype='object')

In [94]:
# Create a pivot table to count the occurrences of each purchase method by gender
purchase_method_by_gender = pd.crosstab(salesdf['gender'], salesdf['purchaseMethod'])

# Remove the index name
purchase_method_by_gender.index.name = "Gender"

print(purchase_method_by_gender)

purchaseMethod  In store  Online  Phone
Gender                                 
F                   8062    4270   1499
M                   7571    4314   1722


In [95]:
# Replace 'F' with 'Female' and 'M' with 'Male' in the index (gender labels)
purchase_method_by_gender = purchase_method_by_gender.rename(index={'F': 'Female', 'M': 'Male'})

# Drop the 'Phone' column
purchase_method_by_gender = purchase_method_by_gender.drop(columns='Phone')

purchase_method_by_gender.index.name = 'Gender (M/F)'

print(purchase_method_by_gender)


purchaseMethod  In store  Online
Gender (M/F)                    
Female              8062    4270
Male                7571    4314


### Query 5: Show monthly total sales

Since 'saleDate' is stored as a dictionary with a key '$date' within each row, we need to access the # value of '$date' and convert it to a datetime object

In [96]:
# Extract the # value of '$date' from the 'saleDate' dictionaries, which are nested within each row. 
# Access the '$date' value within these dictionaries as follows:
salesdf['sales_date'] = salesdf['saleDate'].apply(lambda x: pd.to_datetime(x['$date']))

In [97]:
salesdf.columns

Index(['_id', 'saleDate', 'items', 'storeLocation', 'customer', 'couponUsed',
       'purchaseMethod', 'product_name', 'sales', 'gender', 'sales_date'],
      dtype='object')

In [98]:
salesdf.head()

,_id,saleDate,items,storeLocation,customer,couponUsed,purchaseMethod,product_name,sales,gender,sales_date
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'laptop', 'tags': ['electronics', 'sc...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,laptop,1613.68,M,2017-12-08 21:40:34.527000+00:00
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'envelopes', 'tags': ['stationary', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,envelopes,74.41,M,2017-12-08 21:40:34.527000+00:00
0,{'$oid': '5bd761dcae323e45a93ccff7'},{'$date': '2017-12-08T21:40:34.527Z'},"{'name': 'printer paper', 'tags': ['office', '...",Denver,"{'gender': 'M', 'age': 28, 'email': 'fapra@rut...",False,Phone,printer paper,383.70,M,2017-12-08 21:40:34.527000+00:00
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'printer paper', 'tags': ['office', '...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online,printer paper,431.55,F,2015-05-15 13:43:24.561000+00:00
1,{'$oid': '5bd761dcae323e45a93cd000'},{'$date': '2015-05-15T13:43:24.561Z'},"{'name': 'notepad', 'tags': ['office', 'writin...",London,"{'gender': 'F', 'age': 27, 'email': 'zu@bainku...",False,Online,notepad,141.08,F,2015-05-15 13:43:24.561000+00:00


In [99]:
# Extract the month and year from the 'sales_date' and create a new column
salesdf['year_month'] = salesdf['sales_date'].dt.to_period('M')

# Group by 'year_month' and sum the sales for each month
monthly_total_sales = salesdf.groupby('year_month')['sales'].sum().reset_index()

# Rename the columns for clarity
monthly_total_sales.columns = ['Year-Month', 'Total Sales']

print(monthly_total_sales)


   Year-Month  Total Sales
0     2013-01    196043.76
1     2013-02    147622.59
2     2013-03    140700.06
3     2013-04    162336.67
4     2013-05    179143.37
5     2013-06    142132.02
6     2013-07    176298.23
7     2013-08    130977.22
8     2013-09    137871.99
9     2013-10    173126.73
10    2013-11    148358.38
11    2013-12    174306.96
12    2014-01    125824.02
13    2014-02    141174.41
14    2014-03    193413.50
15    2014-04    132593.08
16    2014-05    166896.80
17    2014-06    143304.56
18    2014-07    184804.60
19    2014-08    167979.81
20    2014-09    159802.92
21    2014-10    149284.84
22    2014-11    180564.85
23    2014-12    139466.83
24    2015-01    222577.01
25    2015-02    189776.15
26    2015-03    178086.94
27    2015-04    132298.50
28    2015-05    167482.69
29    2015-06    149484.34
30    2015-07    149510.47
31    2015-08    154868.63
32    2015-09    135615.80
33    2015-10    175215.34
34    2015-11    158390.42
35    2015-12    166565.02
3

C:\Users\Lyann Thit Nyi Nyi\AppData\Local\Temp\ipykernel_18404\2663829891.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  salesdf['year_month'] = salesdf['sales_date'].dt.to_period('M')
